In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
from sklearn import feature_selection as fs
import sklearn.decomposition as skde
import sklearn.metrics as sklm
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy.stats as ss
import scipy.cluster.hierarchy as sch
from sklearn.linear_model import ElasticNet, Lasso,  Ridge, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import r2_score
import lightgbm as lgb
#import xgboost as xgb

%matplotlib inline

In [ ]:
df_features = pd.read_csv('train_values.csv')
df_labels = pd.read_csv('train_labels.csv')
df = df_features.merge(df_labels, on='row_id')

df.shape

In [ ]:
df = df.drop_duplicates(keep = 'last')
df = df.drop_duplicates(subset = 'row_id', keep = 'last')

df.shape

In [ ]:
#droping state_code -1

indexNames = df[df['state_code'] == -1].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

df.shape

In [ ]:
print(df.isnull().sum())

In [ ]:
#adjust applicant income missing values by median value

df['applicant_income'].fillna(df['applicant_income'].median(), inplace=True)

print(df.isnull().sum())

In [ ]:
#dropping all rows with remaining missing values
indexNames = df[df['population'].isnull()].index 
df.drop(indexNames , inplace=True)

i2 = df[df['minority_population_pct'].isnull()].index 
df.drop(i2 , inplace=True)

i3 = df[df['ffiecmedian_family_income'].isnull()].index 
df.drop(i3 , inplace=True)

i4 = df[df['tract_to_msa_md_income_pct'].isnull()].index 
df.drop(i4 , inplace=True)

i5 = df[df['number_of_owner-occupied_units'].isnull()].index 
df.drop(i5 , inplace=True)

i6 = df[df['number_of_1_to_4_family_units'].isnull()].index 
df.drop(i6 , inplace=True)

df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.to_csv('cleaned.csv', index = False, header = True)

In [ ]:
df2 = pd.read_csv('cleaned.csv')
df2.shape

In [ ]:
df2.describe()

In [ ]:
##Data Exploration Label
#cleaning labels above 13 results in deleting some 129 rows

count = df2[df2['rate_spread'] > 9].index
df2.drop(count , inplace=True)
df2.describe()

In [ ]:
#but improves the skewness to some level

def hist_plot(vals, lab):
    ## Distribution plot of values
    sns.set(style="whitegrid", palette='Blues_r')
    sns.distplot(vals)
    plt.title('Histogram of ' + lab)
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.show()
    
sns.set_style("whitegrid")
hist_plot(df2['rate_spread'], 'Rate Spread')


In [ ]:
#testing squared labels



#FUNCTION FOR returning 0 for ln(0)
from numpy import errstate,isneginf
a = df2['rate_spread']
with errstate(divide='ignore'):
    res = np.log(a)
res[isneginf(res)]=0
hist_plot(res, 'ln RS')


#SQUARE LABELS
#df2['squared_rate_spread'] = np.square(df2['rate_spread'])
#hist_plot(df2['squared_rate_spread'], 'squared_rate_spread')

In [ ]:
#SQUARE LABELS
df2['squared_rate_spread'] = np.square(df2['rate_spread'])
hist_plot(df2['squared_rate_spread'], 'squared_rate_spread')

In [ ]:
#data Exploration for features _ Loan Type

def create_loantype_group (data):
    loan_conditions = [
    (data['loan_type'] == 1 ),
    (data['loan_type'] == 2 ),
    (data['loan_type'] == 3 ),
    (data['loan_type'] == 4 )
    ]
    loan_choices = ['Conventional', 'FHA', 'VA-Guaranteed', 'FHA/RHS']
    data['loan_group'] = np.select(loan_conditions, loan_choices)

create_loantype_group(df2)

In [ ]:
#data Exploration for features _ property Type, loan purpose, occupancy, preapproval

def create_propertytype_group (data):
    property_conditions = [
    (data['property_type'] == 1 ),
    (data['property_type'] == 2 ),
    (data['property_type'] == 3 )
     ]
    property_choices = ['1to4', 'Manufactured', 'MultiFamily']
    data['property_group'] = np.select(property_conditions, property_choices)

create_propertytype_group(df2)

In [ ]:
def create_loanpurpose_group (data):
    conditions = [
    (data['loan_purpose'] == 1 ),
    (data['loan_purpose'] == 2 ),
    (data['loan_purpose'] == 3 )
    ]
    choices = ['Purchase', 'Improvement', 'Refinancing']
    data['loan_purpose_group'] = np.select(conditions, choices)

create_loanpurpose_group(df2)

In [ ]:
def create_occupancy_group (data):
    conditions = [
    (data['occupancy'] == 1 ),
    (data['occupancy'] == 2 ),
    (data['occupancy'] == 3 )
    ]
    choices = ['Owner Occupied', 'not Owner Occupied', 'Not Applicable']
    data['occupancy_group'] = np.select(conditions, choices)

create_occupancy_group(df2)

In [ ]:
def create_preapproval_group (data):
    conditions = [
    (data['preapproval'] == 1 ),
    (data['preapproval'] == 2 ),
    (data['preapproval'] == 3 )
    ]
    choices = ['requested', 'not requested', 'Not Applicable']
    data['preapproval_group'] = np.select(conditions, choices)
    
create_preapproval_group(df2)

In [ ]:
#Applicant Features

def create_applicant_ethnicity_group (data):
    conditions = [
    (data['applicant_ethnicity'] == 1 ),
    (data['applicant_ethnicity'] == 2 ),
    (data['applicant_ethnicity'] == 3 ),
    (data['applicant_ethnicity'] == 4 ),
    (data['applicant_ethnicity'] == 5 )
    ]
    choices = ['Hispanic or Latino', 'NOT Hispanic or Latino', 'Information not provided', 'Not Applicable', 'No co-applicant']
    data['applicant_ethnicity_group'] = np.select(conditions, choices)
    
create_applicant_ethnicity_group(df2)

In [ ]:
def create_applicant_race_group (data):
    conditions = [
    (data['applicant_race'] == 1 ),
    (data['applicant_race'] == 2 ),
    (data['applicant_race'] == 3 ),
    (data['applicant_race'] == 4 ),
    (data['applicant_race'] == 5 ),
    (data['applicant_race'] == 6 ),
    (data['applicant_race'] == 7 ),
    (data['applicant_race'] == 8 )
    ]
    choices = ['American Indian or Alaska Native', 'Asian', 'Black or African American','Native Hawaiian or Other Pacific Islander', 'White', 'Information not provided', 'Not Applicable', 'No co-applicant']
    data['applicant_race_group'] = np.select(conditions, choices)
    
create_applicant_race_group(df2)


In [ ]:
def create_applicant_sex_group (data):
    conditions = [
    (data['applicant_sex'] == 1 ),
    (data['applicant_sex'] == 2 ),
    (data['applicant_sex'] == 3 ),
    (data['applicant_sex'] == 4 ),
    (data['applicant_sex'] == 5 )
    ]
    choices = ['Male', 'Female', 'Information not provided', 'Not Applicable', 'Not Applicable']
    data['applicant_sex_group'] = np.select(conditions, choices)
    
create_applicant_sex_group(df2)

In [ ]:
df2.dtypes

In [ ]:
df2.drop('loan_type', axis = 1, inplace = True)
df2.drop('property_type', axis = 1, inplace = True)
df2.drop('loan_purpose', axis = 1, inplace = True)
df2.drop('occupancy', axis = 1, inplace = True)
df2.drop('preapproval', axis = 1, inplace = True)
df2.drop('applicant_ethnicity', axis = 1, inplace = True)
df2.drop('applicant_race', axis = 1, inplace = True)
df2.drop('applicant_sex', axis = 1, inplace = True)
df2.drop('squared_rate_spread', axis = 1, inplace = True)

In [ ]:
#counts for each object feature
def count_unique(df2, cols):
    for col in cols:
        print('\n' + 'For column ' + col)
        print(df2[col].value_counts())

cat_cols = ['loan_group','property_group','loan_purpose_group','occupancy_group','preapproval_group','applicant_ethnicity_group','applicant_race_group','applicant_sex_group']

count_unique(df2, cat_cols)
df2.shape

In [ ]:
#violin plot
def plot_violin(df2, cols, col_y, title):
    for col in cols:
        sns.set(style="whitegrid")
        sns.set_palette("Set1", n_colors=7, desat=.7)
        sns.violinplot(col, col_y, data=df2)
        plt.xlabel(col) # Set text for the x axis
        plt.ylabel(col_y)# Set text for y axis
        plt.title(title + ' by ' + col)
        plt.show()
        
plot_violin(df2, cat_cols, 'rate_spread', 'rate_spread_I')

In [ ]:
num_cols = ['loan_amount', 'applicant_income', 'population', 'minority_population_pct', 'ffiecmedian_family_income', 'tract_to_msa_md_income_pct', 'number_of_owner-occupied_units', 'number_of_1_to_4_family_units'  ] 

def plot_density_hist(df2, cols, bins = 10, hist = False):
    for col in cols:
        sns.set(style="whitegrid", palette='Blues_r')
        sns.distplot(df[col], bins = bins, rug=True, hist = hist)
        plt.title('Histogram of ' + col) # Give the plot a main title
        plt.xlabel(col) # Set text for the x axis
        plt.ylabel('Frequency')# Set text for y axis
        plt.show()
        
plot_density_hist(df2, num_cols, bins = 20, hist = True)

In [ ]:
sns.set(style="whitegrid", palette='Blues_r')
sns.pairplot(df2[num_cols])

In [ ]:
#correlation matrix

corrs = df2[num_cols].corr()

## Create the hierarchical clustering model
dist = sch.distance.pdist(corrs)   # vector of pairwise distances using correlations
linkage = sch.linkage(dist, method='complete') # Compute the linkages for the clusters
ind = sch.fcluster(linkage, 0.5*dist.max(), 'distance')  # Apply the clustering algorithm

## Order the columns of the correlaton matrix according to the hierarchy
columns = [corrs.columns.tolist()[i] for i in list((np.argsort(ind)))]  # Order the names for the result
corrs_clustered = corrs.reindex(columns) ## Reindex the columns following the heirarchy 

## Correlation Plot
corrs_clustered.style.background_gradient().set_precision(2)

In [ ]:

num_corrs = df2[num_cols].corr()
f,ax = plt.subplots(figsize=(10, 10))
sns.heatmap(num_corrs, annot=True, square=True, linewidths=.1, fmt= '.1f',ax=ax, 
           cmap="RdBu")
plt.show()

Aggregating the Categorial values as below

1. Property Group : combining Manufactured and MultiFamily as Manuf&Multi
2. Occupancy Group : combining Not Applicable and not Owner Occupied as not Applicable&OwnerOccupied
3. Applicant ethnicity Group: combining Not Applicable to Info Not provided as InfoNotPresent
4. Applicant Race Group: combining Not Applicable to Native Hawaian as NH&InfoNotPresent
5. Applicant Sex Group: combining Not applicable to Info Not Provided as InfoNotPresent 

In [ ]:
def plot_box_1(df, col, col_y, title):
    sns.set_style("whitegrid")
    sns.set_palette("Set1", n_colors=7, desat=.7)
    sns.boxplot(col, col_y, data=df)
    plt.xlabel(col) # Set text for the x axis
    plt.ylabel(col_y)# Set text for y axis
    plt.title(title + ' by ' + col)
    plt.show()
    
def plot_violin_1(df, col, col_y, title):
    sns.set_style("whitegrid")
    sns.set_palette("Set1", n_colors=7, desat=.7)
    fig, ax = plt.subplots(figsize=(11,8))
    sns.violinplot(col, col_y, data=df)
    plt.xlabel(col) # Set text for the x axis
    plt.ylabel(col_y)# Set text for y axis
    plt.title(title + ' by ' + col)
    plt.show()

plot_violin_1(df2, 'property_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'occupancy_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_ethnicity_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_race_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_sex_group', 'rate_spread', 'RS')

In [ ]:
#propertygroup aggregations
property_categories = {'1to4':'1to4', 'Manufactured':'Manuf&Multi',
                       'MultiFamily':'Manuf&Multi'}
df2['property_group'] = [property_categories[x] for x in df2['property_group']]


#occupancy group aggregations
occupancy_group_categories = {'Owner Occupied':'Owner Occupied', 'not Owner Occupied':'notApplicable&OwnerOccupied', 'Not Applicable':'notApplicable&OwnerOccupied'}
df2['occupancy_group'] = [occupancy_group_categories[x] for x in df2['occupancy_group']]


#ethnicity group aggregations
applicant_ethnicity_group_categories = {'NOT Hispanic or Latino':'NOT Hispanic or Latino', 'Hispanic or Latino':'Hispanic or Latino', 'Information not provided':'InfoNotPresent', 'Not Applicable':'InfoNotPresent'}
df2['applicant_ethnicity_group'] = [applicant_ethnicity_group_categories[x] for x in df2['applicant_ethnicity_group']]


#applicant_race_group aggregations
applicant_race_group_categories = {'White':'White', 'Black or African American':'Black or African American', 'Information not provided':'InfoNotPresent', 'Asian':'Asian', 'American Indian or Alaska Native':'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander':'Native Hawaiian or Other Pacific Islander', 'Not Applicable':'InfoNotPresent'}
df2['applicant_race_group'] = [applicant_race_group_categories[x] for x in df2['applicant_race_group']]


applicant_sex_group_categories = {'Male':'Male', 'Female':'Female',
                                      'Information not provided':'InfoNotPresent',
                                      'Not Applicable':'InfoNotPresent'}
df2['applicant_sex_group'] = [applicant_sex_group_categories[x] for x in df2['applicant_sex_group']]


In [ ]:
def count_unique(df2, cols):
    for col in cols:
        print('\n' + 'For column ' + col)
        print(df2[col].value_counts())

cat_cols1 = ['property_group','occupancy_group','applicant_ethnicity_group','applicant_race_group','applicant_sex_group']

count_unique(df2, cat_cols1)
df2.shape

In [ ]:
plot_violin_1(df2, 'property_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'occupancy_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_ethnicity_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_race_group', 'rate_spread', 'RS')
plot_violin_1(df2, 'applicant_sex_group', 'rate_spread', 'RS')

attempting to transform the numeric features to log function for even distribution

In [ ]:
hist_plot(df2['loan_amount'],'loan_amount')
df2['ln_loan_amount'] = np.log(df2['loan_amount'])
hist_plot(df2['ln_loan_amount'], 'natural log loan')

In [ ]:
df2['ln_applicant_income'] = np.log(df2['applicant_income'])
df2['ln_population'] = np.log(df2['population'])
df2['ln_ffiecmedian_family_income'] = np.log(df2['ffiecmedian_family_income'])
df2['ln_number_of_owner-occupied_units'] = np.log(df2['number_of_owner-occupied_units'])
df2['ln_number_of_1_to_4_family_units'] = np.log(df2['number_of_1_to_4_family_units'])

In [ ]:
hist_plot(df2['ln_applicant_income'], 'natural log applicant income')
hist_plot(df2['ln_population'], 'natural log population')
hist_plot(df2['ln_ffiecmedian_family_income'], 'natural log mdeian family_incom')
hist_plot(df2['ln_number_of_owner-occupied_units'], 'natural log owner-occupied_units')
hist_plot(df2['ln_number_of_1_to_4_family_units'], 'natural log 1_to_4_family_units')

Model Matrix Preparation
We are predicting the label rate_spread with both categorical and numeric features. Categorical features are one-hot encoded, and joint back with the numeric features.

In [ ]:
Labels = np.array(df2['rate_spread'])

In [ ]:
def encode_string(cat_features):
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder(categories='auto')
    encoded = ohe.fit_transform(cat_features.values.reshape(-1,1)).toarray()
    pdfn = ohe.get_feature_names()
    print(pdfn)
    return encoded

features_cat_cols = ['co_applicant','property_group','loan_purpose_group','occupancy_group','preapproval_group','applicant_ethnicity_group',
                     'applicant_race_group','applicant_sex_group']

Features = encode_string(df2['loan_group'])
for col in features_cat_cols:
    temp = encode_string(df2[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)
print(Features[:2, :])

In [ ]:
features_num_cols = ['ln_loan_amount', 'ln_applicant_income','ln_population','ln_ffiecmedian_family_income','ln_number_of_owner-occupied_units',
                     'ln_number_of_1_to_4_family_units','minority_population_pct','tract_to_msa_md_income_pct']
Features = np.concatenate([Features, np.array(df2[features_num_cols])], axis = 1)
print(Features.shape)
print(Features[:2, :])

Model selection


Eliminate low variance features

We eliminate features with low variance using the scikit-learn VarianceThreshold based function. A p = 0.95 was found to optimize the model.

In [ ]:
print(Features.shape)

## Define the variance threhold and fit the threshold to the feature array.
sel = fs.VarianceThreshold(threshold=(.95 * (1 - .95)))
Features_reduced = sel.fit_transform(Features)
print(sel.get_support())

## Print the support and shape for the transformed features
print(Features_reduced.shape)

Recursive feature elimination

We used the scikit-learn RFECV function to determine which features to retain using a cross validation method, with the ridge regression model:

In [ ]:
## Reshape the Label array
Labels = Labels.reshape(Labels.shape[0],)

## Set folds for nested cross validation
nr.seed(562)
feature_folds = ms.KFold(n_splits=10, shuffle = True)

## Define the model
lin_mod_l2 = linear_model.Ridge()

## Perform feature selection by CV with high variance features only
nr.seed(265)
selector = fs.RFECV(estimator = lin_mod_l2, cv = feature_folds, scoring = 'r2')
selector = selector.fit(Features_reduced, Labels)
print(selector.support_)
print(selector.ranking_)

Features_reduced = selector.transform(Features_reduced)
print(Features_reduced.shape)

Split the Dataset

With the model matrix constructed, we now create randomly sampled training and test data sets in a 7:3 ratio

In [ ]:
nr.seed(265)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
x_train = Features_reduced[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features_reduced[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

Rescale numeric features

We use the StandardScaler to z-score scale the numeric features.

In [ ]:
scaler = preprocessing.StandardScaler().fit(x_train[:,27:])
x_train[:,27:] = scaler.transform(x_train[:,27:])
x_test[:,27:] = scaler.transform(x_test[:,27:])
print(x_train[:2,])

Construct the Regression Models

Different regression models are tested with optimized hyperparameters. We first fit the models with pre-selected parameters to our train data, and fine-tune them recursively. Here we will test Gradient Boosting Regressor, XGboost Regressor and LGB Regressor, the current best-performing algorithms for data science competitions.

In [ ]:
GBoost = GradientBoostingRegressor()
model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves = 32,
                              learning_rate=0.01)

In [ ]:
nr.seed(265)
inside = ms.KFold(n_splits=5, shuffle = True)
nr.seed(562)
outside = ms.KFold(n_splits=5, shuffle = True)

nr.seed(2652)
## Define the dictionary for the grid search and the model object to search on
param_grid = {'n_estimators': [2000, 3000]}

## Perform the grid search over the parameters
gsearch = ms.GridSearchCV(estimator = model_lgb, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)

In [ ]:
gsearch.fit(Features_reduced, Labels)
gsearch.best_params_, gsearch.best_score_

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators= 2000, learning_rate=0.01,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=7, min_samples_split=15, 
                                   loss='ls', random_state = 1)

model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves = 32,
                              learning_rate=0.01, n_estimators=2100, 
                              bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.4,
                              min_data_in_leaf = 5,  
                              feature_fraction_seed=3, bagging_seed=2)

In [ ]:
#Validation function
n_folds = 5

def r2_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train)
    r2 = cross_val_score(model, x_train, y_train, scoring="r2", cv = kf)
    return(r2)

In [ ]:
score = r2_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = r2_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
def rsquare(y, y_pred):
    return r2_score(y, y_pred)

In [ ]:
GBoost.fit(x_train, y_train)
gb_pred = GBoost.predict(x_test)
print(rsquare(y_test, gb_pred))

In [ ]:
model_lgb.fit(x_train, y_train)
lgb_pred = model_lgb.predict(x_test)
print(rsquare(y_test, lgb_pred))

In [ ]:
lgb_pred[lgb_pred < 0] = 0
lgb_pred[lgb_pred > 7] = 7

In [ ]:
def print_metrics(y_true, y_predicted, n_parameters):
    ## First compute R^2 and the adjusted R^2
    r2 = sklm.r2_score(y_true, y_predicted)
    r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)
    
    ## Print the usual metrics and the R^2 values
    print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))
    print('R^2                    = ' + str(r2))
    print('Adjusted R^2           = ' + str(r2_adj))
   
print_metrics(y_test, lgb_pred, 28)

In [ ]:
def resid_qq(y_test, y_score):
    ## first compute vector of residuals. 
    resids = np.subtract(y_test.reshape(-1,1), y_score.reshape(-1,1))
    ## now make the residual plots
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ss.probplot(resids.flatten(), plot = plt)
    ax.get_lines()[0].set_marker('+')
    plt.title('Residuals vs. predicted values')
    plt.xlabel('Predicted values')
    plt.ylabel('Residual')
    
resid_qq(y_test, lgb_pred)

In [ ]:
def resid_plot(y_test, y_score):
    ## first compute vector of residuals. 
    resids = np.subtract(y_test.reshape(-1,1), y_score.reshape(-1,1))
    ## now make the residual plots
    sns.regplot(y_score, resids, fit_reg=False, marker="+", scatter_kws={'alpha':0.5})
    plt.title('Residuals vs. predicted values')
    plt.xlabel('Predicted values')
    plt.ylabel('Residual')

resid_plot(y_test, lgb_pred)